In [2]:
import requests
import pandas as pd
import time
import datetime
from prefect.blocks.system import Secret
import os
from prefect.blocks.system import Secret
from prefect.filesystems import GCS
from prefect_gcp.cloud_storage import GcsBucket
from pathlib import Path

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": ""
}


In [64]:

def write_gcs(path: str):
    gcp_block = GcsBucket.load("lol-datalake")
    gcp_block.upload_from_path(
        from_path=f'{path}',
        to_path='data/'
    )


In [16]:
df = pd.read_csv('./extract_from_gcs/chalengers/2023/4/17/chalenger_17-04-2023.csv', index_col=None)

FileNotFoundError: [Errno 2] No such file or directory: './extract_from_gcs/chalengers/2023/4/17/chalenger_17-04-2023.csv'

In [81]:
response = requests.get('https://br1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1', headers=headers)

In [69]:

def write_local(df: pd.DataFrame) -> str:
    '''Write DataFrame out locally as parquet file'''
    dia = datetime.datetime.now().day
    mes = datetime.datetime.now().month
    ano = datetime.datetime.now().year
    dataset_file = f'chalenger_{dia:02}-{mes:02}-{ano:02}'
    folder_path = './data_flow'
    dir_path = os.path.join(folder_path)
    path = os.path.join(dir_path, dataset_file + '.csv')


    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    df.to_csv(path)
    return path

In [82]:
df = pd.read_json(response.text,orient='records')

In [83]:
write_local(df)

'./data_flow/chalenger_14-04-2023.csv'

In [13]:
df.head()

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid
0,61279367-76f2-34fe-aa72-386a19a90053,RANKED_SOLO_5x5,CHALLENGER,I,oMe8xpXahTZEVIElT6SN80BTa2UPkvWs3-gbleXZ8wx7Ox0,frosty,1908,214,151,True,False,False,False,dobcFjMdScI2VsSaqr8SIIc-4gLafx-qmdEwP45Qm-hOPk...
1,61279367-76f2-34fe-aa72-386a19a90053,RANKED_SOLO_5x5,CHALLENGER,I,iWs_XHNMQHNf0ofFqWOiirYcxh1r90yVgZZ1IDw_dH8sj-...,FURIA Ayu,1906,149,95,True,False,False,False,IzjFQWI5jxL8gV_hY6mJ2KGPIpYGTmwtu67B-FPNsLFw5R...
2,61279367-76f2-34fe-aa72-386a19a90053,RANKED_SOLO_5x5,CHALLENGER,I,1IHjZOFkFSBMwBzKnrzfIUKK74RuoPg4SiRIfNurHId4Re0,Hugatin,1736,141,92,True,False,False,False,rjZxZQEkXegwcdkuqQ8bqdb67U-AILObjLaEwUicnGVLmI...
3,61279367-76f2-34fe-aa72-386a19a90053,RANKED_SOLO_5x5,CHALLENGER,I,XfCU7vQVC3EE8LN-vOiVqVJUBqYYyzVcg7sfj9L8UAGWz3k,Tutsz,1721,213,165,True,False,False,False,Q6H4ALKdhPtAuY5mbYnb8sGJH3cs5SL3_ayZNnqVS5OT_M...
4,61279367-76f2-34fe-aa72-386a19a90053,RANKED_SOLO_5x5,CHALLENGER,I,1gBJyh-HulsRlX8wy3GnqFJB8bH5bAknq9rVSJpZ2pES7Z...,Bvoy3,1640,142,85,True,False,False,False,pYfSgpPZ-aWEgVwoqT6SViee40u0Mw9Uz4Sg0M0SY8O3Jd...


In [85]:
def get_puuid(summoner_id):
    url = f'https://br1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
    response = requests.get(url, headers=headers)
    response_json = response.json()
    print(response_json)
    return response_json['puuid']

In [86]:
puuids = []
for index, row in df.iterrows():
    time.sleep(3)
    puuid = get_puuid(row['summonerId'])
    puuids.append(puuid)

df = df.assign(puuid=puuids)

{'id': 'oMe8xpXahTZEVIElT6SN80BTa2UPkvWs3-gbleXZ8wx7Ox0', 'accountId': 'QWJ1blQAAoqfwGh7FRGVQK4w-WISHbr-ROMQxAv7xKcWVxQ', 'puuid': 'dobcFjMdScI2VsSaqr8SIIc-4gLafx-qmdEwP45Qm-hOPkVIhEP7Wwvq9rNFxZ9NZyNSoA1Xo0qoAQ', 'name': 'frosty', 'profileIconId': 5490, 'revisionDate': 1681492853191, 'summonerLevel': 813}
{'id': 'iWs_XHNMQHNf0ofFqWOiirYcxh1r90yVgZZ1IDw_dH8sj-i_pkLQEg4I0g', 'accountId': '6YIWr_SQ2ATmURA97oqe73bG2RNoy_Zo4HOqXPYe8zv7GdRs6IApaMTK', 'puuid': 'IzjFQWI5jxL8gV_hY6mJ2KGPIpYGTmwtu67B-FPNsLFw5R0bPf5iwohdgRioveL3F__1bOTgALy00A', 'name': 'FURIA Ayu', 'profileIconId': 29, 'revisionDate': 1681334112000, 'summonerLevel': 93}
{'id': '1IHjZOFkFSBMwBzKnrzfIUKK74RuoPg4SiRIfNurHId4Re0', 'accountId': '68tJq8-eEH36gLNBTjUHRMc4pdFi_NH_z0Ka3vEZrW-6V2I', 'puuid': 'rjZxZQEkXegwcdkuqQ8bqdb67U-AILObjLaEwUicnGVLmI2Ppp7kLNXO4Y7446L5eIS5k9IeG7s3yw', 'name': 'Hugatin', 'profileIconId': 1388, 'revisionDate': 1681503954567, 'summonerLevel': 569}
{'id': 'XfCU7vQVC3EE8LN-vOiVqVJUBqYYyzVcg7sfj9L8UAGWz3k', 

/usr/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'Block.load' was never awaited
  return compile(source, filename, mode, flags,


KeyboardInterrupt: 

In [45]:
def get_match_ids(puuid: str) -> list:
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100'
    response = requests.get(url, headers=headers)
    match_ids_list = []
    for id in response.json():
        match_ids_list.append(id)
    return match_ids_list


In [46]:
matches_ids_global_raw = []
for index, row in df.iterrows():
    time.sleep(3)
    matches_ids_especific_summoner = get_match_ids(row['puuid'])
    matches_ids_global_raw.extend(matches_ids_especific_summoner)


In [49]:
print(len(set(matches_ids_global_raw)))

14257


In [ ]:
def get_match_info(match_id: str):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    response = requests.get(url, headers=headers)
    for data in response.json():
                participant_data = {
                'gameId': data['info']['gameId'],
                'gameMode': data['info']['gameMode'],
                'gameType': data['info']['gameType'],
                'id': participant['id'],
                'name': participant['name'],
                'score': participant['score']
            }
    return match_ids_list

In [52]:
import json

with open('response.json') as f:
    dados = json.load(f)

In [18]:
url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_2712884156'
response = requests.get(url, headers=headers)
dados = response.json()
print(dados['info'])


{'gameCreation': 1681406272102, 'gameDuration': 1805, 'gameEndTimestamp': 1681408169958, 'gameId': 2712884156, 'gameMode': 'CLASSIC', 'gameName': 'teambuilder-match-2712884156', 'gameStartTimestamp': 1681406364873, 'gameType': 'MATCHED_GAME', 'gameVersion': '13.7.503.1019', 'mapId': 11, 'participants': [{'allInPings': 0, 'assistMePings': 0, 'assists': 1, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 377, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 9, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 445.1003236896544, 'damageTakenOnTeamPercentage': 0.17550502806974208, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 8, 'dodgeSkillShotsSmallWindow': 1, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1247.3092287860177, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealA

In [19]:
df = pd.DataFrame(columns=['gameId', 'gameMode', 'gameType', 'allInPings', 'assistMePings', 'assists', 'baitPings', 'baronKills', 'basicPings', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'needVisionPings', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'onMyWayPings', 'participantId', 'pentaKills', 'perks', 'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'profileIcon', 'pushPings', 'puuid', 'quadraKills', 'riotIdName', 'riotIdTagline', 'role', 'sightWardsBoughtInGame', 'spell1Casts', 'spell2Casts', 'spell3Casts', 'spell4Casts', 'summoner1Casts', 'summoner1Id', 'summoner2Casts', 'summoner2Id', 'summonerId', 'summonerLevel', 'summonerName', 'teamEarlySurrendered', 'teamId', 'teamPosition', 'timeCCingOthers', 'timePlayed', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead', 'totalUnitsHealed', 'tripleKills', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken', 'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills', 'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced', 'win', 'timestamp'])

game_id = dados['info']['gameId']
game_mode = dados['info']['gameMode']
game_type = dados['info']['gameType']
participantes = dados['info']['participants']

lista_participantes = []
for p in participantes:
    d = {'gameId': game_id, 'gameMode': game_mode, 'gameType': game_type}
    d.update(p)
    lista_participantes.append(d)
df['timestamp'] = pd.Timestamp.now().strftime('%Y-%m-%d')

In [32]:
list = []
for col in df.columns:
    list.append(col)
print(list)

['gameId', 'gameMode', 'gameType', 'allInPings', 'assistMePings', 'assists', 'baitPings', 'baronKills', 'basicPings', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDama

In [31]:
df['timestamp'] = pd.Timestamp.now().strftime('%Y-%m-%d')

In [50]:
df = pd.read_csv('/home/guilhermefmk/workspace/DEzoomcamp_Final_project/data_flow/match_ids_2023_04_17_match_ids_17-04-2023.csv', index_col=None)

In [38]:
df.head()

,match_id
0,BR1_2693061582
1,BR1_2702594033
2,BR1_2693883344
3,BR1_2694130193
4,BR1_2713145977


In [59]:
df_main = pd.DataFrame(columns=['gameId', 'gameMode', 'gameType', 'patch', 'gameDuration', 'gameCreation','allInPings', 'assistMePings', 'assists', 'baitPings', 'baronKills', 'basicPings', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'needVisionPings', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'onMyWayPings', 'participantId', 'pentaKills', 'perks', 'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'profileIcon', 'pushPings', 'puuid', 'quadraKills', 'riotIdName', 'riotIdTagline', 'role', 'sightWardsBoughtInGame', 'spell1Casts', 'spell2Casts', 'spell3Casts', 'spell4Casts', 'summoner1Casts', 'summoner1Id', 'summoner2Casts', 'summoner2Id', 'summonerId', 'summonerLevel', 'summonerName', 'teamEarlySurrendered', 'teamId', 'teamPosition', 'timeCCingOthers', 'timePlayed', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead', 'totalUnitsHealed', 'tripleKills', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken', 'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills', 'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced', 'win', 'timestamp'], dtype=str)
    

In [18]:
def get_match_data(match_id: str, header: dict):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    response = requests.get(url, headers=header)
    dados = response.json()
    return dados

In [19]:
def struct_dict(dados) -> list:
    game_id = dados['info']['gameId']
    game_mode = dados['info']['gameMode']
    game_type = dados['info']['gameType']
    patch = dados['info']['gameVersion']
    participantes = dados['info']['participants']
    game_creation = dados['info']['gameCreation']
    game_start = dados['info']['gameStartTimestamp']
    game_end = dados['info']['gameStartTimestamp']
    game_duration = dados['info']['gameDuration']

    lista_participantes = []
    for p in participantes:
        d = {
                'gameId': game_id, 
                'gameMode': game_mode, 
                'gameType': game_type, 
                'patch': patch,
                'gameStart': game_start,
                'gameEnd': game_end,
                'gameDuration': game_duration,
                'gameCreation': game_creation
            }
        p.pop('challenges', None)
        p.pop('perks', None)
        d.update(p)
        lista_participantes.append(d)
    return lista_participantes

In [51]:

def transform(df: pd.DataFrame) -> pd.DataFrame:
    df['gameCreation'] = pd.to_datetime(df['gameCreation'], unit='ms')
    df['gameDuration'] = pd.to_datetime(df['gameDuration'], unit='s')
    return df


In [60]:
contador = 0
for index, row in df.iterrows():
    contador += 1
    if contador < 10:
        time.sleep(1)
        json_response = get_match_data(row['match_id'], headers)
        dados_participantes = struct_dict(json_response)
        df_partida = pd.DataFrame(dados_participantes)
        df_main = pd.concat([df_main, df_partida], ignore_index=True)
    else: 
        break
df_main['timestamp'] = pd.Timestamp.now().strftime('%Y-%m-%d')


In [64]:
df_main.head()

,gameId,gameMode,gameType,patch,gameDuration,gameCreation,allInPings,assistMePings,assists,baitPings,...,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,timestamp,gameStart,gameEnd
0,2693061582,CLASSIC,MATCHED_GAME,13.4.494.1549,1719,1678239551076,0,1,2,0,...,0,0,24,2,4,8,False,2023-04-18,1.678240e+12,1.678241e+12
1,2693061582,CLASSIC,MATCHED_GAME,13.4.494.1549,1719,1678239551076,1,5,4,2,...,0,0,29,10,3,18,False,2023-04-18,1.678240e+12,1.678241e+12
2,2693061582,CLASSIC,MATCHED_GAME,13.4.494.1549,1719,1678239551076,0,9,7,0,...,0,0,21,0,2,9,False,2023-04-18,1.678240e+12,1.678241e+12
3,2693061582,CLASSIC,MATCHED_GAME,13.4.494.1549,1719,1678239551076,0,1,7,0,...,0,1,24,1,4,8,False,2023-04-18,1.678240e+12,1.678241e+12
4,2693061582,CLASSIC,MATCHED_GAME,13.4.494.1549,1719,1678239551076,0,3,13,0,...,0,0,79,11,14,31,False,2023-04-18,1.678240e+12,1.678241e+12


In [91]:
df_main.dtypes

gameId          object
gameMode        object
gameType        object
patch           object
gameDuration    object
                 ...  
wardsPlaced     object
win             object
timestamp       object
gameStart       object
gameEnd         object
Length: 130, dtype: object

In [85]:
import json

for column in df_main.columns:
    try:
        json.dumps(df_main[column])
    except TypeError:
        print(f"Column {column} contains non-serializable data.")

Column gameId contains non-serializable data.
Column gameMode contains non-serializable data.
Column gameType contains non-serializable data.
Column patch contains non-serializable data.
Column gameDuration contains non-serializable data.
Column gameCreation contains non-serializable data.
Column allInPings contains non-serializable data.
Column assistMePings contains non-serializable data.
Column assists contains non-serializable data.
Column baitPings contains non-serializable data.
Column baronKills contains non-serializable data.
Column basicPings contains non-serializable data.
Column bountyLevel contains non-serializable data.
Column challenges contains non-serializable data.
Column champExperience contains non-serializable data.
Column champLevel contains non-serializable data.
Column championId contains non-serializable data.
Column championName contains non-serializable data.
Column championTransform contains non-serializable data.
Column commandPings contains non-serializable

In [65]:
df_main.to_json()

'{"gameId":{"0":2693061582,"1":2693061582,"2":2693061582,"3":2693061582,"4":2693061582,"5":2693061582,"6":2693061582,"7":2693061582,"8":2693061582,"9":2693061582,"10":2702594033,"11":2702594033,"12":2702594033,"13":2702594033,"14":2702594033,"15":2702594033,"16":2702594033,"17":2702594033,"18":2702594033,"19":2702594033,"20":2693883344,"21":2693883344,"22":2693883344,"23":2693883344,"24":2693883344,"25":2693883344,"26":2693883344,"27":2693883344,"28":2693883344,"29":2693883344,"30":2694130193,"31":2694130193,"32":2694130193,"33":2694130193,"34":2694130193,"35":2694130193,"36":2694130193,"37":2694130193,"38":2694130193,"39":2694130193,"40":2713145977,"41":2713145977,"42":2713145977,"43":2713145977,"44":2713145977,"45":2713145977,"46":2713145977,"47":2713145977,"48":2713145977,"49":2713145977,"50":2705562056,"51":2705562056,"52":2705562056,"53":2705562056,"54":2705562056,"55":2705562056,"56":2705562056,"57":2705562056,"58":2705562056,"59":2705562056,"60":2713539817,"61":2713539817,"62":2

In [84]:
df_main.select_dtypes(include=['float64']).columns

Index([], dtype='object')

In [79]:
df_main.columns.value_counts().sum()

130

In [83]:
df_main = df_main.astype(str)

In [4]:
my_dict = {'key1': 'value1', 'key2': 'value2'}
new_dict = {'key2': 'new_value', 'key3': 'value3'}
my_dict.update(new_dict)